In [4]:
!pip install streamlit torch torchaudio noisereduce soundfile speechbrain



In [5]:
app_code = """
import streamlit as st
import torchaudio
import torch
import noisereduce as nr
from speechbrain.pretrained import EncoderClassifier
import torchaudio.transforms as T
import numpy as np
import soundfile as sf
import tempfile

st.title("🎙️ Voice Biometrics with ECAPA-TDNN")

@st.cache_resource
def load_model():
    return EncoderClassifier.from_hparams(
        source="speechbrain/spkrec-ecapa-voxceleb",
        savedir="pretrained_models/spkrec-ecapa-voxceleb"
    )

model = load_model()

def get_embedding(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    if sample_rate != 16000:
        waveform = T.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    reduced = nr.reduce_noise(y=waveform.squeeze().numpy(), sr=16000)
    waveform = torch.tensor(reduced).unsqueeze(0)
    vad = T.Vad(sample_rate=16000)
    waveform = vad(waveform)
    waveform = waveform / waveform.abs().max()
    min_len = 16000
    if waveform.shape[1] < min_len:
        pad = min_len - waveform.shape[1]
        waveform = torch.nn.functional.pad(waveform, (0, pad))
    embedding = model.encode_batch(waveform)
    return embedding.squeeze().numpy()

option = st.radio("Choose input method:", ["📁 Upload file"])

if option == "📁 Upload file":
    uploaded = st.file_uploader("Upload audio file (wav, mp3, ogg)", type=["wav", "mp3", "ogg"])
    if uploaded:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
            tmp.write(uploaded.read())
            tmp.flush()
            emb = get_embedding(tmp.name)
            st.success(" Audio processed.")
            st.write("🔎 Embedding shape:", emb.shape)
"""

with open("app.py", "w") as f:
    f.write(app_code)


In [6]:
!pip install pyngrok


In [8]:
!ngrok authtoken 2yFBWdkkoA91HKE61meyHWPNgzk_2qtukSGNvDk1HnJzkXS89


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
from pyngrok import ngrok
import subprocess
import time

# Kill any previous streamlit processes
!pkill -f streamlit

# Create tunnel on port 8501 (Streamlit's default)
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

# 3. Run your Streamlit app in background
command = "streamlit run app.py"
process = subprocess.Popen(command.split())

# Give streamlit some time to start
time.sleep(5)

Public URL: NgrokTunnel: "https://6efe-34-9-118-252.ngrok-free.app" -> "http://localhost:8501"
